# **Predict Future Sales**

Final project for "How to win a data science competition" Coursera course
This challenge serves as final project for the "How to win a data science competition" Coursera course.

In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company.

We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

We have the following 6 data sets for this project

* test
* train
* shops
* items[](http://)
* item categorty
* submission

In [ ]:
# import libraries 
import pandas as pd
import numpy as np
import warnings 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
from string import ascii_letters
import plotly.express as px
import sys
import gc
import pickle
import time

In [ ]:

df_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
df_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
df_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
df_shop = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
df_item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
df_item = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
# View head of df_train data set
print(df_train.head())

In [ ]:
# View the shape and size of the train data set
print("Shape of the data Set is :  ",df_train.shape)
print("Size of the data Set is  :  ",df_train.size)

In [ ]:
# Verify the result, you can see the Dtype of 'date' is changed to datetime64
print(df_train.info())

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])

In [ ]:
# Verify the result, you can see the Dtype of 'date' is changed to datetime64
print(df_train.info())

In [ ]:
# This is a Graph to show the Item ID sold by month 
plt.figure(figsize = (18,10))
plt.title('Items Sold Per Month', fontsize=22)
ax = sns.barplot(x='date_block_num', y="item_id", data=df_train, estimator=np.sum)
ax = ax.set(xlabel='Date Blocks', ylabel='Item ID')
plt.xticks(rotation=45)
plt.show() 

In [ ]:
# This is a Graph to show the Item ID sold by Shop Id 
plt.figure(figsize = (18,10))
plt.title('Items Sold Shop ID')
ax = sns.barplot(x='shop_id', y="item_id", data=df_train, estimator=np.mean)
ax = ax.set(xlabel='Shop Id', ylabel='Item ID')
plt.xticks(rotation=45)
plt.show() 

In [ ]:
#View the headers of the data set 
df_shop.head(2)

In [ ]:
df_shop['shop_name']

In [ ]:
#Find out the unique count of shop id 
df_shop['shop_id'].count()

In [ ]:
# I have learnt this from one of the notebooks there are dupicates and this needs correction 
df_shop.loc[df_shop['shop_id']==0, 'shop_id'] = 57
df_shop.loc[df_shop['shop_id']==1, 'shop_id'] = 58
df_shop.loc[df_shop['shop_id']==10, 'shop_id'] = 11

In [ ]:
duplicate_shop_names = {
    df_shop.loc[df_shop['shop_id']==57, 'shop_name'].values[0]:df_shop.loc[df_shop['shop_id']==57, 'shop_name'].values[1],
    df_shop.loc[df_shop['shop_id']==58, 'shop_name'].values[0]:df_shop.loc[df_shop['shop_id']==58, 'shop_name'].values[1],
    df_shop.loc[df_shop['shop_id']==11, 'shop_name'].values[0]:df_shop.loc[df_shop['shop_id']==11, 'shop_name'].values[1] 
}

In [ ]:
#df_shop.head(100)
df_shop = df_shop.drop_duplicates(subset='shop_id')

In [ ]:

# Count of unique shop id after data cleansing 
df_shop['shop_id'].count()

In [ ]:
df_shop['city'] = df_shop['shop_name'].str.split(' ').apply(lambda x: x[0])
# We will concat one of rows which has some white spaces names in shop name 
df_shop.loc[df_shop.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
# We will remove special characters in the city 
df_shop.loc[df_shop['city'] == '!Якутск', 'city'] = 'Якутск'
# View first 2 rows of the data set
df_shop.head(2)

In [ ]:
# This is a Graph to show the Item ID sold by Shop Id 
plt.figure(figsize = (18,10))
plt.title('# of stores in the city', fontsize=24)
ax = sns.barplot(x='city', y="shop_id", data=df_shop, estimator=np.sum)
ax = ax.set(xlabel='Stores', ylabel='City')
plt.xticks(rotation=45)

plt.show() 

In [ ]:
# View # of unique shops in the data set
print("Number of unique shops: {}".format(len(df_shop['shop_id'].unique())))

In [ ]:
# view the unique shop ID in train data set
df_train['shop_id'].nunique()

In [ ]:
# Shop id have to be corrected in train data set as well. 

## Shopnames for 0 and 57 are the same so changing shop_id 0 to 57
df_train.loc[df_train['shop_id']==0, 'shop_id'] = 57

## Shopnames for 1 and 58 are the same so changing shop_id 1 to 58
df_train.loc[df_train['shop_id']==1, 'shop_id'] = 58

## Shopnames for 10 and 11 are the same so changing shop_id 00 to 11
df_train.loc[df_train['shop_id']==10, 'shop_id'] = 11

In [ ]:
# Verify the unique shop ID in train data set
df_train['shop_id'].nunique()

In [ ]:
# view first 2 rows of the dataset
df_item.head(2)

In [ ]:
# Find the unique number of item names in the dataset
df_item['item_name'].nunique()

In [ ]:
# View the first 2 records of item category data set 
df_item_cat.head(2)

In [ ]:
# Create a new column by name category and split the same by, instead of -
df_item_cat['category']=df_item_cat['item_category_name'].str.split('-')

In [ ]:
# Create a new column by name type and split the first part of the category
df_item_cat['type']=df_item_cat['category'].apply(lambda x: x[0].strip())

In [ ]:
# Create a new column by name sub type and split the second part of the category 
df_item_cat['sub_type'] = df_item_cat['category'].apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())

In [ ]:
# now our data set looks good and we will remove 2 columns
df_item_cat.drop(['item_category_name','category'], inplace=True, axis=1)

In [ ]:
# view the first 4 records of the data set
df_item_cat.head(4)

In [ ]:
# lets find out now the count of unique type and sub type 
print("    Unique Types: ",df_item_cat['type'].nunique())
print("Unique Sub Types: ",df_item_cat['sub_type'].nunique())

In [ ]:
# This is a Graph to show the Item Category ID and the item type name 

plt.figure(figsize = (18,10))
plt.title('# of types of items category wise', fontsize=24)
ax = sns.barplot(x='item_category_id', y='type', data=df_item_cat, estimator=np.sum)
ax = ax.set(xlabel='Category ID', ylabel='Type')
plt.xticks(rotation=45)

plt.show() 

In [ ]:
# Create a new data frame that will merge data from all the above data sets
# we will call this df as ,master data frame 
# we will merge Train and item on item item_id
master_data = pd.merge(df_train, df_item, how='left',on='item_id')

In [ ]:
# We will merge the newly create master data with item category on item category id 
master_data = pd.merge(master_data, df_item_cat, how='left', on='item_category_id')

# We will further merge the data with Shop data set on shop id 
master_data = pd.merge(master_data, df_shop, how='left',on='shop_id')

# lets look at the size and shape of newly created data frame master data
print(" Shape: ", master_data.shape)
print("  Size: ", master_data.size)

In [ ]:
# Grouping data by month, shop id and Item ID
month = master_data.groupby(['date_block_num','shop_id','item_id']).agg({'item_price':'mean','item_cnt_day':'sum'}).reset_index()



In [ ]:
# Combining the data frames

# Merge data with item 
month = pd.merge(month,df_item,how='left',on='item_id')

# Merge data with item categories 
month = pd.merge(month, df_item_cat,how='left',on='item_category_id')

# merge data with shops

month = pd.merge(month, df_shop, how='left',on='shop_id')

In [ ]:
# lets look at the size and shape of month data 
print(" Shape: ", month.shape)
print("  Size: ", month.size)

In [ ]:
# Create a new colums called total sales 
month['tota_sales'] = month['item_price']*month['item_cnt_day']

In [ ]:
# Plot a line graph to show the monthly sales 
plt.figure(figsize = (18,10))
plt.title('# Monthly sales', fontsize=24)
ax = sns.lineplot(x='date_block_num', y='tota_sales', data=month, estimator=np.sum)
ax = ax.set(xlabel='Date Block', ylabel='Total Sales')
plt.xticks(rotation=45)

plt.show() 

In [ ]:
# Create a column called month
month['month']= month['date_block_num'].apply(lambda month: (month+1)%12)

In [ ]:
# Dummifying categorical columns 

# First let us work on shop 
month = pd.concat([month, pd.get_dummies(month['shop_id'],drop_first=True, prefix='shop_')], axis=1)

# Let us work on type 

month = pd.concat([month, pd.get_dummies(month['type'],drop_first=True, prefix='type')], axis=1)

# Finally work on sub type 

month = pd.concat([month, pd.get_dummies(month['sub_type'],drop_first=True, prefix='sub_type')], axis=1)

# lets look at the size and shape of month data 
print(" Shape: ", month.shape)
print("  Size: ", month.size)

In [ ]:
# Getting the names of feature columns for shop, type and sub_type 

# Defining shop column 

shop_col = [col for col in month.columns if 'shop__' in col]

# Defining type column 

type_col = [col for col in month.columns if 'type_' in col]

# Defining sub_type column 

sub_type_col = [ col for col in month.columns if 'sub_type_' in col]

In [ ]:
# Defining features and target 
features = ['month', 'shop_id','item_id', 'item_price'] + type_col + sub_type_col
target = ['item_cnt_day']

In [ ]:
# Import for splitting data 
from sklearn.model_selection import train_test_split

# Setup for feature and target variable
X_feature = month[features].fillna(value=0)
Y_target = month[target].fillna(value=0)

# Split and train and test 
X_train, X_test, Y_train, Y_test = train_test_split(X_feature, Y_target, test_size=0.3, random_state=0)



In [ ]:
# Fitting Linear Regression Model 

# Get lr function by importing from sklearn 
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# Fit the lr model on the training data 

lr.fit(X_train,Y_train)

In [ ]:
# view the score on the data set
lr.score(X_test,Y_test)

In [ ]:
# add the date_block_num and month column 
df_test['date_block_num'] = 34
df_test['month'] = 11

In [ ]:
item_price = month[['item_id', 'item_price']].groupby('item_id')['item_price'].mean().reset_index()

In [ ]:
# Merging the test data frame 

# Merge with item price 
df_test = pd.merge(df_test, item_price,how='left',on='item_id')

# Merge with items 
df_test = pd.merge(df_test, df_item ,how='left',on='item_id')

# Merge with item categories  
df_test = pd.merge(df_test, df_item_cat ,how='left',on='item_category_id')

# Merge with shop 
df_test = pd.merge(df_test, df_shop ,how='left',on='shop_id')

In [ ]:
# Dummifying the categorical columns to get the test on shop, type and sub_type 

# Creating dummies for shop 
df_test = pd.concat([df_test, pd.get_dummies(df_test['shop_id'], drop_first=True, prefix='shop_')], axis=1)

## Creating dummies for type
df_test = pd.concat([df_test, pd.get_dummies(df_test['type'], drop_first=True, prefix='type')], axis=1)

## Creating dummies for sub_type
df_test = pd.concat([df_test, pd.get_dummies(df_test['sub_type'], drop_first=True, prefix='sub_type')], axis=1)

In [ ]:
# Getting the names of the feature columns

# Collecting shop feature names
df_test_shop_col = [col for col in df_test.columns if 'shop__' in col]

# Collecting type feature names
df_test_type_col = [col for col in df_test.columns if 'type_' in col]

# Collecting sub_type feature names
df_test_sub_type_col = [col for col in df_test.columns if 'sub_type_' in col]

In [ ]:
# Creating features for test data 
df_test_features = ['month', 'item_id' + 'shop_id', 'item_price'] + df_test_type_col + df_test_sub_type_col



In [ ]:
# getting common features 
common_features = list(set(features) & set(df_test_features)) 
print(f"Number of common features: {len(common_features)}")

In [ ]:
# verify if the item_price is available in common features 
'item_price' in common_features

In [ ]:
# Preparing data for modeling

# Setting feature and target variables
X = month[common_features].fillna(value=0)
y = month[target].fillna(value=0)

## Splitting train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
## Getting LR function
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

## Fitting on training data
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
# 
df_test = df_test.fillna(0)

In [ ]:
# Predict
lr.predict(df_test[common_features])

In [ ]:
df_test['preds'] = lr.predict(df_test[common_features])

In [ ]:
# Create a file for submission 
preds = df_test[['ID', 'preds']]
preds.columns = ['ID', 'item_cnt_month']
preds

In [ ]:
# Saving the submission dataframe
preds.to_csv('my_submission.csv', index=False)

In [ ]:
# Coming soon will work on the Second model using XG Boost

#from xgboost import XGBRegressor
#xg = XGBRegressor()

## Fitting on training data
#xg.fit(X_train, y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
from sklearn.pipeline import Pipeline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU
from keras.utils import np_utils

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [ ]:
decision_tree.predict(df_test[common_features])

In [ ]:
df_test['predsdt'] = decision_tree.predict(df_test[common_features])

In [ ]:
predsdt = df_test[['ID', 'preds']]
predsdt.columns = ['ID', 'item_cnt_month']
predsdt

In [ ]:
predsdt.to_csv('my_submission1.csv', index=False)

In [ ]:
RFR = RandomForestRegressor(n_estimators = 100)
RFR.fit(X_train,Y_train)

print('Train set mse:', mean_squared_error(Y_train, RFR.predict(X_train)))
print('Test set mse:', mean_squared_error(Y_test, RFR.predict(X_test)))
print('Test set score:', RFR.score(X_train,Y_train))

In [ ]:
prediction = RFR.predict(X_test)

In [ ]:
prediction = list(map(round, prediction))

In [ ]:
df_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(df_submission.shape)
df_submission.head()